In [ ]:
import collections
import numpy as np
import pickle


In [ ]:
experiments = ['BM25',
               'BERT_base',
               ]

metrics = ['RaB', 'ARaB'] 
methods = ['tf', 'bool']


qry_bias_paths = {}
for metric in metrics:
    qry_bias_paths[metric] = {}
    for exp_name in experiments:
        qry_bias_paths[metric][exp_name] = {}
        for _method in methods:
            qry_bias_paths[metric][exp_name][_method] = 'data/%s_run_bias_%s_%s.pkl' % (exp_name, _method, metric)
        

queries_gender_annotated_path = "resources/queries_gender_annotated.csv"

at_ranklist = [5, 10, 20, 30, 40]

In [ ]:
qry_bias_perqry = {}

for metric in metrics:
    qry_bias_perqry[metric] = {}
    for exp_name in experiments:
        qry_bias_perqry[metric][exp_name] = {}
        for _method in methods:
            _path = qry_bias_paths[metric][exp_name][_method]
            print (_path)
            with open(_path, 'rb') as fr:
                qry_bias_perqry[metric][exp_name][_method] = pickle.load(fr)



In [ ]:
queries_effective = {}
with open(queries_gender_annotated_path, 'r') as fr:
    for line in fr:
        vals = line.strip().split(',')
        qryid = int(vals[0])
        qrytext = ' '.join(vals[1:-1])
        qrygender = vals[-1]
        if qrygender == 'n':
            queries_effective[qryid] = qrytext
len(queries_effective)

In [ ]:
eval_results_bias = {}
eval_results_feml = {}
eval_results_male = {}

for metric in metrics:
    eval_results_bias[metric] = {}
    eval_results_feml[metric] = {}
    eval_results_male[metric] = {}
    for exp_name in experiments:
        eval_results_bias[metric][exp_name] = {}
        eval_results_feml[metric][exp_name] = {}
        eval_results_male[metric][exp_name] = {}
        for _method in methods:
            eval_results_bias[metric][exp_name][_method] = {}
            eval_results_feml[metric][exp_name][_method] = {}
            eval_results_male[metric][exp_name][_method] = {}
            for at_rank in at_ranklist:
                _bias_list = []
                _feml_list = []
                _male_list = []

                for qryid in queries_effective.keys():
                    if qryid in qry_bias_perqry[metric][exp_name][_method][at_rank]:
                        _bias_list.append(qry_bias_perqry[metric][exp_name][_method][at_rank][qryid][0])
                        _feml_list.append(qry_bias_perqry[metric][exp_name][_method][at_rank][qryid][1])
                        _male_list.append(qry_bias_perqry[metric][exp_name][_method][at_rank][qryid][2])
                    else:
                        pass
                        #print ('missing', metric, exp_name, _method, at_rank, qryid)

                eval_results_bias[metric][exp_name][_method][at_rank] = np.mean([(_male_x-_feml_x) for _male_x, _feml_x in zip(_male_list, _feml_list)])
                eval_results_feml[metric][exp_name][_method][at_rank] = np.mean(_feml_list)
                eval_results_male[metric][exp_name][_method][at_rank] = np.mean(_male_list)




In [ ]:

for metric in metrics:
    print (metric)
    for at_rank in at_ranklist:
        for _method in methods:
            for exp_name in experiments:
                print ("%25s\t%2d %5s\t%f\t%f\t%f" % (exp_name, at_rank, _method, eval_results_bias[metric][exp_name][_method][at_rank], eval_results_feml[metric][exp_name][_method][at_rank], eval_results_male[metric][exp_name][_method][at_rank]))
            print ()
        print ("==========")